In [1]:
import csv
import json
import nltk
import string
import random
import math
import re
import os
from pprint import pprint
from copy import deepcopy
from collections import defaultdict, Counter, namedtuple, OrderedDict
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /Users/deedy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/deedy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [335]:
def read_to_lines(fname) -> str:
    with open(fname, 'r') as f:
        data = f.read()
    lines = [d for d in data.split('\n') if len(d)]
    return lines

In [336]:
# https://github.com/dwyl/english-words
# DICT = 'words_alpha.txt'
# https://github.com/zeisler/scrabble
DICT = 'dictionary_proper.txt'
all_words = read_to_lines(DICT)
len(all_words)

172820

In [337]:
# Maybe use words with frequencies to better predict the letter distribution - not sure if uniformly 
# sampling all words is the right approach
five_words = [w for w in all_words if len(w) == 5]
len(five_words)

8636

In [338]:
five_word_chars = Counter([c for w in five_words for c in w])
global_five_freq = {let: freq for let, freq in five_word_chars.most_common()}
five_word_chars.most_common()

[('s', 4511),
 ('e', 4456),
 ('a', 3842),
 ('o', 2871),
 ('r', 2834),
 ('i', 2546),
 ('l', 2371),
 ('t', 2261),
 ('n', 1957),
 ('d', 1676),
 ('u', 1633),
 ('c', 1444),
 ('p', 1353),
 ('y', 1347),
 ('m', 1291),
 ('h', 1177),
 ('g', 1063),
 ('b', 1035),
 ('k', 917),
 ('f', 770),
 ('w', 679),
 ('v', 466),
 ('z', 229),
 ('x', 203),
 ('j', 173),
 ('q', 75)]

In [805]:
from string import ascii_lowercase
pos_counts = []
for i in range(0, 5):
    common = Counter([w[i] for w in five_words]).most_common()
    if len(common) != 26:
        available = set([c[0] for c in common])
        notavail = [c for c in ascii_lowercase if not c in available]
        for x in notavail:
            common.append((x, 0))
    pos_counts.append(common)
for x in range(26):
    print([(p[x][0], f'{p[x][1]*100/len(five_words):.02f}%') for p in pos_counts])
pos_totals

[('s', '12.23%'), ('a', '17.13%'), ('a', '9.84%'), ('e', '19.74%'), ('s', '31.21%')]
[('c', '7.70%'), ('o', '15.46%'), ('r', '9.16%'), ('a', '7.64%'), ('e', '10.78%')]
[('b', '6.82%'), ('e', '12.31%'), ('i', '8.60%'), ('t', '7.23%'), ('y', '10.49%')]
[('p', '6.54%'), ('i', '11.19%'), ('o', '7.84%'), ('i', '6.68%'), ('d', '6.74%')]
[('t', '6.35%'), ('u', '9.16%'), ('n', '7.30%'), ('l', '6.16%'), ('r', '6.06%')]
[('a', '5.75%'), ('r', '7.47%'), ('l', '6.76%'), ('n', '6.07%'), ('t', '5.56%')]
[('d', '5.29%'), ('l', '5.89%'), ('e', '6.54%'), ('r', '5.28%'), ('n', '4.21%')]
[('m', '5.14%'), ('h', '4.40%'), ('u', '5.14%'), ('o', '5.27%'), ('a', '4.12%')]
[('f', '4.98%'), ('n', '2.77%'), ('t', '5.13%'), ('s', '4.10%'), ('l', '3.99%')]
[('r', '4.85%'), ('t', '1.92%'), ('s', '3.95%'), ('k', '3.76%'), ('h', '2.95%')]
[('g', '4.81%'), ('y', '1.91%'), ('m', '3.90%'), ('d', '3.53%'), ('o', '2.58%')]
[('l', '4.64%'), ('p', '1.75%'), ('d', '3.17%'), ('c', '3.32%'), ('k', '2.11%')]
[('h', '3.87%'), ('

[8636, 8636, 8636, 8636, 8636]

In [825]:
pos_distros[0]

defaultdict(int,
            {'s': 1056,
             'c': 665,
             'b': 589,
             'p': 565,
             't': 548,
             'a': 497,
             'd': 457,
             'm': 444,
             'f': 430,
             'r': 419,
             'g': 415,
             'l': 401,
             'h': 334,
             'w': 296,
             'k': 206,
             'n': 200,
             'e': 192,
             'o': 181,
             'v': 161,
             'j': 134,
             'i': 118,
             'u': 117,
             'y': 94,
             'q': 53,
             'z': 51,
             'x': 13})

In [910]:
pos_distros = []
for p in pos_counts:
    pos_distro = defaultdict(int)
    for char, count in p:
        pos_distro[char] = count
    pos_distros.append(pos_distro)

def sort_maximal_position(word):
    score = [0]*26
    for i, c in enumerate(word):
        score[ord(c)-ord('a')] = pos_distros[i][c]
    return -sum(score)
sorted(five_words, key=sort_maximal_position)[:10]
NONPOS_WEIGHT = 0.5
[(c, sort_maximal_position_with_nonpos(c, sort=False, debug=False)) for c in sorted(five_words, key=sort_maximal_position_with_nonpos)[:10]]

[('tares', (7218, 8870, -11653.0)),
 ('lares', (7071, 9127, -11634.5)),
 ('rales', (6882, 9316, -11540.0)),
 ('rates', (6741, 9347, -11414.5)),
 ('nares', (6870, 8914, -11327.0)),
 ('tales', (7011, 8614, -11318.0)),
 ('dares', (7127, 8376, -11315.0)),
 ('cares', (7335, 7936, -11303.0)),
 ('pares', (7235, 7945, -11207.5)),
 ('arles', (6126, 10072, -11162.0))]

In [908]:
cands_s = ['bibbs', 'biffs', 'binds', 'bints', 'bisks', 'bitts', 'dings', 'dinks', 'dints', 'disks', 'djins', 'finds', 'finks', 'fists', 'gifts', 'gimps', 'ginks', 'gists', 'highs', 'hinds', 'hints', 'hists', 'imids', 'jibbs', 'jiffs', 'jinks', 'jinns', 'jisms', 'kinds', 'kings', 'kinks', 'kists', 'kiths', 'knits', 'miffs', 'miggs', 'minds', 'minks', 'mints', 'mists', 'mitts', 'nighs', 'pimps', 'pings', 'pinks', 'pints', 'piths', 'pyins', 'shims', 'shins', 'ships', 'shits', 'shivs', 'sibbs', 'sifts', 'sighs', 'signs', 'simps', 'sings', 'sinhs', 'sinks', 'skids', 'skims', 'skins', 'skips', 'skits', 'snibs', 'snips', 'snits', 'spiks', 'spins', 'spits', 'spivs', 'swigs', 'swims', 'swiss', 'thins', 'tiffs', 'tings', 'tints', 'twigs', 'twins', 'twits', 'whids', 'whigs', 'whims', 'whins', 'whips', 'whits', 'wimps', 'winds', 'wings', 'winks', 'wisps', 'wists', 'zings']
pos_distros = [defaultdict(int) for i in range(Wordle.N)]
for w in cands_s:
    for i, c in enumerate(w):
        pos_distros[i][c] += 1

unknown_places = set()
known_places = ['_' for i in range(Wordle.N)]
for i, p in enumerate(pos_distros):
    distro = [x for x in p.values() if x]
    if len(distro) == 1:
        known_places[i] = [k for k in p.keys()][0]
        pos_distros[i] = defaultdict(int)
    else:
        unknown_places.add(i)
print(''.join(known_places), unknown_places)

NONPOS_WEIGHT = 0.25
# When weight is 1, we basically count non positional hits the same way as positional hits
# weight should be 0 < w < 1, but its unclear what the optimal value is
def sort_maximal_position_with_nonpos(word, sort=True, debug=False):
    score = [0]*26
    nonpos_score = [0]*26
    for i, c in enumerate(word):
        score[ord(c)-ord('a')] = pos_distros[i][c]
        nonpos = 0
        if not c in known_places:
            for j in range(Wordle.N):
                if i != j:
                    nonpos += pos_distros[j][c]
        nonpos_score[ord(c)-ord('a')] = nonpos
    sortscore = -(sum(score) + NONPOS_WEIGHT*sum(nonpos_score))
    if sort:
        return sortscore
    if debug:
        return ([(chr(i + ord('a')), v) for i, v in enumerate(score) if v],
                [(chr(i+ ord('a')), v) for i, v in enumerate(nonpos_score) if v], sortscore)
    else:
        return (sum(score), sum(nonpos_score), sortscore)
print(unknown_places, pos_distros)
[(c, sort_maximal_position_with_nonpos(c, sort=False)) for c in sorted(five_words, key=sort_maximal_position_with_nonpos)[:10]]


____s {0, 1, 2, 3}
{0, 1, 2, 3} [defaultdict(<class 'int'>, {'b': 6, 'd': 5, 'f': 3, 'g': 4, 'h': 4, 'i': 1, 'j': 5, 'k': 6, 'm': 7, 'n': 1, 'p': 6, 's': 28, 't': 7, 'w': 12, 'z': 1}), defaultdict(<class 'int'>, {'i': 62, 'j': 1, 'm': 1, 'n': 4, 'y': 1, 'h': 12, 'k': 5, 'p': 4, 'w': 6}), defaultdict(<class 'int'>, {'b': 3, 'f': 6, 'n': 30, 's': 10, 't': 4, 'i': 34, 'm': 4, 'g': 5}), defaultdict(<class 'int'>, {'b': 4, 'f': 4, 'd': 9, 't': 23, 'k': 12, 'g': 11, 'n': 10, 'p': 9, 'h': 6, 'm': 5, 'v': 2, 's': 1}), defaultdict(<class 'int'>, {})]


[('singe', (131, 59, -145.75)),
 ('minty', (122, 72, -140.0)),
 ('mints', (122, 71, -139.75)),
 ('pinta', (121, 74, -139.5)),
 ('pinto', (121, 74, -139.5)),
 ('pints', (121, 74, -139.5)),
 ('hints', (119, 79, -138.75)),
 ('bints', (121, 68, -138.0)),
 ('dints', (120, 70, -137.5)),
 ('sinew', (120, 68, -137.0))]

In [888]:
for w in range(11):
    NONPOS_WEIGHT = w/10.0
    print(f'Weight {NONPOS_WEIGHT:.02f}')
    print([(c, sort_maximal_position_with_nonpos(c)) for c in sorted(five_words, key=sort_maximal_position_with_nonpos)[:5]])


Weight 0.00
[('singe', -131.0), ('mints', -122.0), ('minty', -122.0), ('bints', -121.0), ('pinta', -121.0)]
Weight 0.10
[('singe', -136.9), ('minty', -129.2), ('mints', -129.1), ('pinta', -128.4), ('pinto', -128.4)]
Weight 0.20
[('singe', -142.8), ('minty', -136.4), ('mints', -136.2), ('pinta', -135.8), ('pinto', -135.8)]
Weight 0.30
[('singe', -148.7), ('minty', -143.6), ('mints', -143.3), ('pinta', -143.2), ('pinto', -143.2)]
Weight 0.40
[('singe', -154.6), ('minty', -150.8), ('hints', -150.6), ('pinta', -150.6), ('pinto', -150.6)]
Weight 0.50
[('singe', -160.5), ('hints', -158.5), ('minty', -158.0), ('pinta', -158.0), ('pinto', -158.0)]
Weight 0.60
[('skint', -167.0), ('hints', -166.4), ('singe', -166.4), ('pinta', -165.4), ('pinto', -165.4)]
Weight 0.70
[('skint', -182.0), ('stink', -180.5), ('sting', -178.39999999999998), ('night', -174.8), ('hints', -174.3)]
Weight 0.80
[('skint', -197.0), ('stink', -196.0), ('sting', -193.60000000000002), ('think', -189.4), ('night', -189.2)]
We

In [857]:
[x for x in pos_distros[4].values() if x]

[96]

In [357]:
test_lets = 'aesor' # arose, oreas, seora
test_lets = 'nutil' # unlit, until
test_lets = 'dcympei' # unlit, until
for w in five_words:
    if len([1 for l in test_lets if l in w]) == 5:
        print(w)


dicey
imped
medic


In [358]:
[w for w in five_words if w[1] == 'i' and w[3]=='e' and w[4] == 'd' and len([1 for l in 'asrountlcybkv' if l in w]) == 0]

['fifed', 'fixed', 'hided', 'mimed', 'mixed', 'piped', 'wifed', 'wiped']

In [359]:
class Wordle:
    N = 5
    MAX_GUESSES = 6
    # tile
    NOTHING = 0
    GUESS_WRONG_SPOT = 1
    GUESS_RIGHT_SPOT = 2
    # states
    PLAYING = 0
    SOLVED = 1
    UNSOLVED = 2
    
    def __init__(self, word, verbose=True):
        global five_words
        self.all_words = five_words
        assert word in self.all_words
        assert len(word) == Wordle.N
        self._word = word.lower()
        self.guesses = []
        self.state = Wordle.PLAYING
        self.verbose = verbose
    
    def emojify(clue):
        pclue = []
        for c in clue:
            if c == Wordle.NOTHING:
                pclue.append('⬛')
            elif c == Wordle.GUESS_WRONG_SPOT:
                pclue.append('🟨')
            elif c == Wordle.GUESS_RIGHT_SPOT:
                pclue.append('🟩')
            else:
                assert False
        return ''.join(pclue)
    
    # Encoding: 0, nothing, 1 guess wrong spot, 2 guess right spot
    def guess(self, guess):
        if self.state == Wordle.SOLVED:
            if self.verbose:
                print('Already solved!')
            return None, self.state
        if self.state == Wordle.UNSOLVED:
            if self.verbose:
                print('Already lost!')
            return None, self.state
        guess = guess.lower()
        assert len(guess) == Wordle.N
        assert guess in self.all_words
        self.guesses.append(guess)
        clue = [Wordle.NOTHING] * Wordle.N
        for i, g in enumerate(guess):
            if self._word[i] == g:
                clue[i] = Wordle.GUESS_RIGHT_SPOT
            elif g in self._word:
                clue[i] = Wordle.GUESS_WRONG_SPOT
        if self.verbose:
            print(Wordle.emojify(clue))
            
        if len([1 for c in clue if c == Wordle.GUESS_RIGHT_SPOT]) == Wordle.N:
            if self.verbose:
                print(f'Solved! - {guess}')
            self.state = Wordle.SOLVED
        elif len(self.guesses) >= Wordle.MAX_GUESSES:
            if self.verbose:
                print(f'Lost! - {self.guesses}')
            self.state = Wordle.UNSOLVED
        
        return clue, self.state

w = Wordle('gorge')
w.guess('great')
w.guess('fairy')
w.guess('champ')
w.guess('crazy')
w.guess('query')
# w.guess('brash')
w.guess('gorge')

🟩🟨🟨⬛⬛
⬛⬛⬛🟨⬛
⬛⬛⬛⬛⬛
⬛🟨⬛⬛⬛
⬛⬛🟨🟨⬛
🟩🟩🟩🟩🟩
Solved! - gorge


([2, 2, 2, 2, 2], 1)

In [455]:
word = 'radon' # random.choice(five_words)
w = Wordle(word)
w.guess('arose')

🟨🟨🟨⬛⬛


([1, 1, 1, 0, 0], 0)

In [850]:
DEFAULT_SETTINGS = {
        'use_conditional': True,
        'non_strict': True,
        'use_pos': True,
    }

def solve_wordle(wordle, settings=DEFAULT_SETTINGS, debug=False):
    clues = []
    for i in range(Wordle.MAX_GUESSES):
        chosen, cands, numcands = solve_guesser(clues, settings=settings, debug=debug)
        if debug:
            print(f'Choosing [{chosen}]. Total {numcands} candidates: {cands}...')
        clue, state = w.guess(chosen)
        if clue:
            clues.append((chosen, clue))
        if state == Wordle.SOLVED:
            if debug:
                print('Woohoo! Solver solved it!')
            return True, cands
        elif state == Wordle.UNSOLVED:
            if debug:
                print('Oh no, it beat the solver :(')
            return False, cands

In [901]:
# We need all the indexes of a letter occuring in a word. 
# For words like [leper] we will fail because we only consider the first position of 'e' with 'leper'.index('e')
def indexall(w, let):
    return set([i for i, c in enumerate(w) if c == let])


def is_guessable_word(w, word_right_place, in_word_wrong_place, not_in_word, debug=False):
    for let in word_right_place.keys():
        if not let in w:
            if debug:
                print(f'Letter [{let}] needs to be in word (in right order)!')
            return False
        for ix in word_right_place[let]:
            if not ix in indexall(w, let):
                if debug:
                    print(f'Letter [{let}] needs to be in word at [{word_right_place[let]}] - {ix} {w}, {indexall(w, let)}')
                return False
    for let in in_word_wrong_place.keys():
        if not let in w:
            if debug:
                print(f'Letter [{let}] needs to be in word!')
            return False
        for ix in indexall(w, let):
            if ix in in_word_wrong_place[let]:
                if debug:
                    print(f'Letter [{let}] needs to be in word and NOT at [{in_word_wrong_place[let]}]')
                return False
    for let in not_in_word:
        if let in w:
            if debug:
                print(f'Letter [{let}] cannot be in word!')
            return False
    return True
    
DEFAULT_SETTINGS = {
        'use_conditional': True,
        'non_strict': True,
        'use_pos': True,
    }


def solve_guesser(clues, settings=DEFAULT_SETTINGS, debug=False):
#     if len(clues) == 0:
#         return 'cares', five_words[:5], len(five_words)
    fword = ['_'] * Wordle.N
    not_in_word = set()
    in_word_wrong_place = defaultdict(set)
    word_right_place = defaultdict(set)
    prev_guesses = set([w for w, _ in clues])
    for w, clue_res in clues:
        for i in range(len(clue_res)):
            if clue_res[i] == Wordle.NOTHING:
                not_in_word.add(w[i])
            elif clue_res[i] == Wordle.GUESS_WRONG_SPOT:
                in_word_wrong_place[w[i]].add(i)
            elif clue_res[i] == Wordle.GUESS_RIGHT_SPOT:
                fword[i] = w[i]
                word_right_place[w[i]].add(i)
#                 if w[i] in in_word_wrong_place:
#                     del in_word_wrong_place[w[i]]
            else:
                assert False
    if debug:
        word_format = ''.join(fword)
        wrong_place = [(x, y) for x, y in in_word_wrong_place.items()]
        not_word = ''.join(not_in_word)
        print(f'Right: {word_format} Wrong: {wrong_place} Absent: {not_word}')
    cands = []
    for w in five_words:
        if is_guessable_word(w, word_right_place, in_word_wrong_place, not_in_word):
            cands.append(w)
            
    char_freq_heuristic = defaultdict(int)
    if settings['use_conditional']:
        # Heuristic should be conditional on what is left!
        # TODO: This doesnt need to happen when exploring - can save compute
        conditional_five_freq = {let: freq for let, freq in Counter([l for c in cands for l in c]).most_common()}
        char_freq_heuristic = conditional_five_freq
    else:
        char_freq_heuristic = global_five_freq
    def sortfn(word):
        # Distinct letters num times appear
        return -sum([char_freq_heuristic[c] for c in set(word)])
    sortcands = sorted(cands, key=sortfn)
    chosen = [cand for cand in sortcands if cand not in prev_guesses][0]
    
    # Explore (non-strict candidates) if
    # - we have 3 of the letters right 
    # - more cands than guesses, we go non-strict to reduce the set. 
    # - not last guess
    new_musts = [set() for x in range(Wordle.N)]
    for ix in range(Wordle.N):
        for c in cands:
            new_musts[ix].add(c[ix])
    for ix, cond in enumerate(new_musts): 
        if len(cond) == 1:
            letter = list(cond)[0]
            word_right_place[letter].add(ix)
    places_known = set([x for setx in word_right_place.values() for x in setx])
    guess_left = Wordle.MAX_GUESSES - len(clues)
    if settings['non_strict'] and len(places_known) >= 0 and len(cands) > guess_left and guess_left != 1:
        unknown_places = set([ix for ix in range(Wordle.N) if not ix in places_known])
        conditional_unknown_freq = defaultdict(int)
        conditional_pos_freq = [defaultdict(int) for i in range(Wordle.N)]
        
        total_unknown_freq = 0
        for c in cands: 
            if c in prev_guesses:
                continue
            for i, l in enumerate(c):
                if i in unknown_places:
                    if not l in not_in_word: # and not i in word_right_place[l]:# and not l in in_word_wrong_place:
                        conditional_unknown_freq[l] += 1
                        conditional_pos_freq[i][l] += 1
                        total_unknown_freq += 1
#         if debug and guess_left == 3:
#             import pdb; pdb.set_trace()
        if total_unknown_freq > 0:
            def exploresortfn(word):
                # Distinct letters num times appear
                return -sum([conditional_unknown_freq[c] for c in set(word)])
            def exploresortfn_pos(word):
                score = [0]*26
                for i, c in enumerate(word):
                    score[ord(c)-ord('a')] = conditional_pos_freq[i][c]
                return -sum(score)
            def sort_maximal_position_with_nonpos(word, sort=True):
                score = [0]*26
                nonpos_score = [0]*26
                for i, c in enumerate(word):
                    score[ord(c)-ord('a')] = conditional_pos_freq[i][c]
                    nonpos_score[ord(c)-ord('a')] = conditional_unknown_freq[c] - conditional_pos_freq[i][c]
                sortscore = -(sum(score) + 0.5*sum(nonpos_score))
                if sort:
                    return sortscore
                return ([(chr(i + ord('a')), v) for i, v in enumerate(score) if v],
                        [(chr(i+ ord('a')), v) for i, v in enumerate(nonpos_score) if v], sortscore)
            
            sortfn = exploresortfn
            if settings['use_pos']:
                sortfn = sort_maximal_position_with_nonpos
            sortcands_expl = sorted(five_words, key=sortfn)
            max_val = sortfn(sortcands_expl[0])
            sortcands_expl = [x for x in sortcands_expl if sortfn(x) == max_val]
            
            def exploresortfn2(word):
                score = 0
                for i, c in enumerate(word):
                    if c in new_musts[i] and len(new_musts[i]) > 1:
                        if c in word_right_place and i in word_right_place[c]:
                            score += 1
                        score += 1
                return -score
            if len(sortcands_expl) > 0:
#                 if debug: len(max_cands) < 100:
#                     print(f'double explore cands ', [(c, exploresortfn2(c)) for c in max_cands])
                sortcands_expl = sorted(sortcands_expl, key=exploresortfn2)
                max_val2 = exploresortfn2(sortcands_expl[0])
                sortcands_expl = [x for x in sortcands_expl if exploresortfn2(x) == max_val2]
            
            def exploresortfn3(word):
                score = 0
                for i, c in enumerate(word):
                    if c in new_musts[i] and len(new_musts[i]) > 1:
                        if c in word_right_place:
                            score += 1
                return -score
            if len(sortcands_expl) > 0:
                sortcands_expl = sorted(sortcands_expl, key=exploresortfn3)
                
                max_val3 = exploresortfn3(sortcands_expl[0])
                sortcands_expl = [x for x in sortcands_expl if exploresortfn3(x) == max_val3]
            if debug: 
                print('Inferred conditions ', [''.join(m) for m in new_musts])
                cond_probs = [(x, y) for x, y in conditional_unknown_freq.items() if y]
                print(f'Conditional ({len(cond_probs)}):{cond_probs}\nCands ({len(cands)}): {cands[:100]}...\n')
                print(f'Explore cands ({len(sortcands_expl)}): {[(c, sortfn(c), exploresortfn2(c), exploresortfn3(c)) for c in sortcands_expl[:10]]}')
        else:
            import pdb; pdb.set_trace()
            sortcands_expl = sortcands
        chosen_cs = [cand for cand in sortcands_expl if cand not in prev_guesses]
        chosen = chosen_cs[0]
        
        
    
    return chosen, sortcands[:5], len(sortcands)
     
w = Wordle('wived')
solve_guesser([('arose', [1, 1, 0, 1, 1])], debug=True)

Right: _____ Wrong: [('a', {0}), ('r', {1}), ('s', {3}), ('e', {4})] Absent: o
Inferred conditions  ['wpmbrcefslgydtnh', 'wpmesyah', 'xwbzcejytksnavprflg', 'pmreldna', 'wrcslydi']
Conditional (23):[('b', 4), ('a', 69), ('r', 72), ('e', 69), ('s', 71), ('c', 4), ('d', 4), ('l', 7), ('n', 4), ('k', 3), ('y', 4), ('f', 3), ('g', 3), ('h', 4), ('m', 4), ('p', 5), ('j', 1), ('t', 3), ('v', 2), ('x', 1), ('z', 1), ('w', 6), ('i', 1)]
Cands (69): ['bares', 'baser', 'bears', 'cares', 'dares', 'dears', 'earls', 'earns', 'escar', 'eskar', 'eyras', 'fares', 'fears', 'gears', 'hares', 'hears', 'lares', 'laser', 'lears', 'mares', 'maser', 'nares', 'nears', 'pares', 'pears', 'races', 'rages', 'rajes', 'rakes', 'rales', 'rapes', 'rares', 'rased', 'raser', 'rases', 'rates', 'raves', 'raxes', 'razes', 'reads', 'reals', 'reams', 'reaps', 'rears', 'resaw', 'resay', 'rheas', 'saber', 'safer', 'sager', 'saker', 'saner', 'saver', 'sawer', 'sayer', 'sears', 'serac', 'serai', 'seral', 'sewar', 'shear', 'smear

('rales', ['earls', 'lares', 'laser', 'lears', 'rales'], 69)

In [916]:
word =  'rases' # random.choice(five_words)
print(f'Instantiating Wordle with the word [{word}]:')
w = Wordle(word)
solve_wordle(w, debug=True)

Instantiating Wordle with the word [rases]:
Right: _____ Wrong: [] Absent: 
Inferred conditions  ['xudiowqbzcejythksnavpmrflg', 'xudiowqbzcejythksnavpmrflg', 'xudiowqbzcejythksnavpmrflg', 'xudiowbzcejythksnavpmrflg', 'xudiowqbzceythksnavpmrflg']
Conditional (26):[('a', 3842), ('h', 1177), ('e', 4456), ('d', 1676), ('l', 2371), ('i', 2546), ('r', 2834), ('g', 1063), ('b', 1035), ('c', 1444), ('k', 917), ('f', 770), ('t', 2261), ('m', 1291), ('p', 1353), ('s', 4511), ('y', 1347), ('o', 2871), ('n', 1957), ('u', 1633), ('v', 466), ('z', 229), ('x', 203), ('w', 679), ('j', 173), ('q', 75)]
Cands (8636): ['aahed', 'aalii', 'aargh', 'abaca', 'abaci', 'aback', 'abaft', 'abaka', 'abamp', 'abase', 'abash', 'abate', 'abbas', 'abbes', 'abbey', 'abbot', 'abeam', 'abele', 'abets', 'abhor', 'abide', 'abler', 'ables', 'abmho', 'abode', 'abohm', 'aboil', 'aboma', 'aboon', 'abort', 'about', 'above', 'abris', 'abuse', 'abuts', 'abuzz', 'abyes', 'abysm', 'abyss', 'acari', 'acerb', 'aceta', 'ached', 'ache

(True, ['rases'])

Solver is deterministic. Note, all these results depend on the word list used! 
Is it a truly solvable game for the English language? 
Nope, it is not. Consider the case of 'gorge' where in 3 guesses (arose, rotge, corge), we know the last 4 letters. Yet, there are 4 candidates and 3 guesses left so by pigeonhole principle, there will be one of those words which we cannot possibly deterministically guess in 6 rounds. The adversarial Evil Wordle demonstrates this!

Currently the solver failed for gorge! It has three deficiencies:
 - FIXED It's a strict solver. Every guess *must* satisfy the conditions of the previous guesses. 
     - Can this exploring algorithm get smarter with pruning? 
 - The candidate sorting character frequency heuristic is not position-based but global. When we don't know where the letters are, fallback on some global positional prior to shape our guesses. 
 - FIXED The character frequency heuristic is not conditional on the candidates left. 

 
(Using dictionary_proper)
 - Without conditional + non-strict: 86.3% of 1000 samples. Failures: ['hight', 'feels', 'jolly', 'byres', 'wills', 'paver', 'troth', 'wrier', 'pares', 'sucks', 'bozos', 'belts', 'kanes', 'trite', 'wifes', 'konks', 'berry', 'sewer', 'evens', 'raved', 'hexes', 'calfs', 'vends', 'vexed', 'eaved', 'warks', 'zones', 'birrs', 'meter', 'rider', 'wries', 'wilts', 'loses', 'loves', 'mamma', 'keefs', 'goofs', 'woofs', 'slaws', 'kevel', 'rises', 'bores', 'gyves', 'fakes', 'jewed', 'fines', 'gales', 'sinks', 'brees', 'bower', 'fries', 'howfs', 'marts', 'dated', 'agaze', 'roses', 'zaxes', 'wafer', 'flams', 'hates', 'willy', 'catch', 'zonks', 'baffs', 'skill', 'haves', 'silos', 'kavas', 'bones', 'bloom', 'razed', 'coyly', 'sakis', 'laded', 'clags', 'titis', 'swabs', 'mater', 'forks', 'gager', 'geeks', 'dewed', 'wussy', 'botts', 'mamma', 'navvy', 'stats', 'hards', 'babes', 'wurst', 'fever', 'forky', 'wists', 'preps', 'safes', 'wifes', 'subas', 'wends', 'taker', 'byres', 'sices', 'brows', 'foods', 'lulus', 'swoop', 'zooms', 'woody', 'howes', 'fires', 'ninny', 'veils', 'marks', 'sissy', 'gated', 'graze', 'gears', 'caver', 'fuses', 'direr', 'found', 'gayer', 'nexus', 'caved', 'jibes', 'bandy', 'gores', 'slogs', 'jager', 'hills', 'meter', 'lolly', 'jetes', 'feyly', 'forge', 'jinns', 'laver', 'witch']
 - With conditional, strict: 91.5% of 1000 samples. Failures: ['maxes', 'pelts', 'pally', 'jills', 'vasal', 'dexes', 'wends', 'berry', 'tents', 'mints', 'youse', 'joked', 'sooks', 'brays', 'nears', 'flaxy', 'eaved', 'rates', 'dreed', 'rimes', 'skees', 'molly', 'hitch', 'moves', 'hangs', 'braza', 'punts', 'taxer', 'masks', 'undue', 'goony', 'ionic', 'waler', 'yores', 'mises', 'laves', 'pangs', 'punts', 'konks', 'sipes', 'cells', 'lanes', 'yipes', 'river', 'swage', 'yirth', 'spall', 'fezes', 'weals', 'hunch', 'golds', 'swage', 'zones', 'wards', 'works', 'viers', 'hiker', 'ended', 'tarts', 'rared', 'fazed', 'soars', 'lawed', 'vasts', 'metes', 'waler', 'zarfs', 'coxed', 'motts', 'saids', 'waxes', 'bloop', 'loner', 'goxes', 'works', 'widdy', 'rexes', 'rexes', 'sojas', 'jocks', 'doled', 'mossy', 'wiver', 'pears', 'noons']
 - With conditional + non-strict: 97.4% of 1000 samples. Failures: ['wests' - 2, 'mises' - 2, 'spars' - 4, 'reifs', 'coses', 'saxes', 'coded', 'sulus', 'sakes', 'undue', 'rexes', 'sakes', 'coyly', 'razer', 'tells', 'vests', 'vills', 'riser', 'mutts', 'wests', 'seres', 'sooks', 'tetra', 'kudus', 'rares', 'noses']
 - With conditional + non-strict + bug fix: 98.5% of 1000 samples. Failures: ['wests', 'mises', 'coses', 'saxes', 'coded', 'sakes', 'undue', 'rexes', 'sakes', 'coyly', 'vests', 'vills', 'wests', 'seres', 'rares']
 - With conditional + non-strict + bug fix: 98.2% of 1000 samples. Failures: ['vills', 'vails', 'mimer', 'songs', 'sipes', 'jives', 'samps', 'vamps', 'piped', 'sided', 'takes', 'terns', 'razes', 'waves', 'javas', 'texts', 'palls', 'jowls']
 - With conditional + non-strict + bug fix for conditional freq wrong place: 99.1% of 1000 samples. Failures: ['vills' - 5!, 'nanas' - 2 (naans), 'tetra' - 2 (terra), 'jives' - 5, 'butts' - 2 (busts), 'vesta' - 3 (steak, testa), 'razes' - 7, 'waves' - 5, 'texts' - 2 (sexts)]
 - Test on all samples 107 down from 150 fails: ['veers', 'vests', 'sises', 'widdy', 'sires', 'tetra', 'rajes', 'toted', 'bases', 'razes', 'wises', 'river', 'nones', 'sales', 'fifes', 'seels', 'razed', 'zests', 'netts', 'sates', 'gases', 'eaves', 'galls', 'sumps', 'sades', 'sulus', 'sells', 'coded', 'texts', 'torts', 'salal', 'rarer', 'zills', 'tater', 'rares', 'sanes', 'kiddy', 'sakes', 'sages', 'berry', 'janes', 'rared', 'tents', 'tilts', 'lulus', 'tates', 'sores', 'riper', 'jiver', 'cases', 'saves', 'firer', 'jades', 'perea', 'testa', 'rases', 'watts', 'saxes', 'sails', 'zaxes', 'seeps', 'sikes', 'sabes', 'sooks', 'raser', 'sizes', 'vills', 'lases', 'riser', 'tight', 'toots', 'wowed', 'oozes', 'sixes', 'tests', 'saver', 'eaved', 'nines', 'tutus', 'gages', 'eases', 'sills', 'seers', 'feres', 'sones', 'jefes', 'razer', 'setts', 'fifed', 'rexes', 'seeds', 'faxed', 'parer', 'loxes', 'seres', 'vases', 'rider', 'rises', 'matts', 'sinks', 'waves', 'soles', 'loses', 'coses', 'faded', 'dated', 'sines']



 
Numbers 
 - words_alpha.txt: There are 15,918 total 5-letter words in this set (4.3% of all 370,103 words)
 - dictionary_proper.txt (Scrabble): 8,636 total 5-letter words in this set (4.997% of all 172,819 words)
 - Another way to evaluate could be to see how many total attempts you need to solve, and look at the distribution. 


Bug fixes:
 - .index() not working correctly in candidate gen because didn't take into account letter appearing twice
 - letter in right place has a bug! 'zaman'
 - Two fixes: Never guess a word you've guessed before both in the explore and candidate path!
 - For 'tesla' - 4 candidates with no distinguishable freqs! ['setal', 'steal', 'stela', 'tesla'], or spars - use the guessed words wrong place as conditional freq 
 - When we guess it in the right place, we shouldn't remove all the wrong places the letter could be because it may expand our set "scans" had [khans, scans], and when we guessed khans, we removed the other places a is not, and it brought back azans, and we lost. 
 - For the bug fix for 'tesla' above, don't keep adding cond freqs for each candidate!
 
(Using words_alpha)
Failure to solve for: gorge, wares, shoos, gains, sware, varia, casks, jades, worms, slaws, zaman, finns, vinny
After conditional word heiristic - ['pokey', 'lelia', 'wrier', 'ricer', 'stubb', 'kikes', 'neist', 'panne', 'falla', 'cleck', 'parde', 'zaire', 'mater', 'siver', 'alain'], 
After non-strict exploration (with threshold 2 right): 
 - From initial list (gorge, wares, ...): Only ['slaws' - 3 cands, 'jades' - 3 cands, 'vinny' - 4] remain
 - From second list (pokey, lelia,...): Only ['ricer' - 2, 'falla' - 2] remain

After non-strict exploration (with threshold 0 right): 
 - From both lists ['varia' - 2, 'jades' - 4, 'vinny' - 3]


Evaluation:
 - Global character frequency heuristic: Couldn't solve for 133 out of 1000 random samples (86.7% Success rate)
 - Conditional character frequency heuristic: Couldn't solve for 100 out of 1000 random samples (90.0% Success rate)
 - Non-strict solve: Couldn't solve for 46 out of 1000 random samples (95.4% Success rate)

In [130]:
clues, solve_guesser(clues)

{'l': 0, 'e': 3, 'r': 4} defaultdict(<class 'set'>, {}) {'s', 'g', 'u', 't', 'a', 'i', 'o'}
leper: False


([('arose', [0, 1, 0, 0, 1]),
  ('liter', [2, 0, 0, 2, 2]),
  ('luger', [2, 0, 0, 2, 2])],
 ([], 0))

In [287]:
is_guessable_word('gorge', {}, {'r': {1}, 'o': {2}}, set({'s', 'a'}), debug=True)
is_guessable_word('zaman', {'a': {1}, 'n': {4}}, {}, set({'r', 'c', 'e', 's', 'h', 'l', 'j', 'y', 'u', 'd', 't', 'i', 'o'}), debug=True)

True

In [204]:
tmpcands= []
for w in five_words:
    if is_guessable_word(w, {}, {}, set('arose')):
        tmpcands.append(w)
random.choices(tmpcands, k=10)

['cubby',
 'milly',
 'unbid',
 'unbud',
 'tucky',
 'lungi',
 'undug',
 'wingy',
 'bluff',
 'chili']

In [918]:
fails = []
samples = []
samples += ['vills', 'oozed', 'sabes', 'sixes', 'bises', 'jived', 'winds', 'wills', 'fifes', 'joker', 'baked', 'zaxes', 'eaves', 'rears', 'river', 'jakes', 'molls', 'dozes', 'zills', 'sises', 'wises', 'pills', 'jiber', 'vanes', 'punks', 'lolly', 'eases', 'jills', 'waxes', 'welly', 'zests', 'jowed', 'seeps', 'jokes', 'saxes', 'sills', 'saves', 'wives', 'fuses', 'faded', 'pekes', 'solos', 'jibes', 'poops', 'winks', 'eaved', 'sumps', 'sipes', 'putts', 'riper', 'bases', 'gills', 'oozes', 'haded', 'vests', 'polos', 'fazes', 'fifed', 'rover', 'sizes', 'vexed', 'songs']
samples += ['wived', 'eases', 'jades', 'razes', 'kinks', 'vills', 'sixes', 'samps', 'rears', 'fuses', 'rases', 'wills', 'sects', 'lulus', 'wanes', 'wives', 'wakes', 'pawer', 'coses', 'septs', 'goxes', 'razed', 'raper', 'tarts', 'caves', 'pumps', 'waves', 'faxed', 'bises', 'kanes', 'river', 'emeer', 'ricer', 'sices', 'vines', 'volts', 'sikes', 'oozed', 'gages', 'zarfs', 'wades', 'rares', 'faves', 'zests', 'sacks', 'sises', 'wises', 'emmer', 'stats', 'fazes', 'rared', 'vairs', 'jakes', 'jills', 'lases', 'sears', 'dived', 'cozes', 'vasts', 'vests', 'sipes', 'zaxes', 'gases', 'jaded', 'jests', 'farer', 'barbs', 'fixes', 'javas', 'gaged', 'vases', 'pases', 'rarer', 'janes', 'faded', 'texts', 'bizes', 'sizes', 'veeps', 'tests', 'hazes', 'sexes', 'vexer', 'warts', 'raver', 'fifes', 'lolly', 'keets', 'eaves', 'jiber', 'japes', 'fazed', 'justs', 'setts', 'waxes']
samples += ['raver', 'wanes', 'lases', 'gaged', 'rared', 'texts', 'wades', 'eaves', 'wived', 'sears', 'jakes', 'wives', 'sipes', 'caves', 'goxes', 'kinks', 'vills', 'rases', 'cozes', 'faxed', 'oozed', 'wakes', 'zaxes', 'tests', 'veeps', 'zarfs', 'ricer', 'javas', 'wises', 'gases', 'fifes', 'raper', 'sices', 'bises', 'coses', 'sikes', 'rarer', 'setts', 'jiber', 'fixes', 'vines', 'volts', 'vases', 'farer', 'zests', 'waxes', 'jills', 'pumps', 'fazed', 'waves', 'janes', 'sacks', 'jests', 'vairs', 'rears', 'sixes', 'vasts', 'wills', 'fazes', 'vests', 'sexes', 'bizes', 'hazes', 'barbs', 'sises', 'jades', 'lolly', 'sizes', 'dived', 'eases', 'river']
samples += ['wanes', 'lases', 'gaged', 'texts', 'wades', 'eaves', 'wived', 'jakes', 'wives', 'sipes', 'caves', 'goxes', 'kinks', 'vills', 'rases', 'cozes', 'faxed', 'oozed', 'wakes', 'pawer', 'zaxes', 'tests', 'zarfs', 'ricer', 'javas', 'wises', 'gases', 'fifes', 'raper', 'sices', 'bises', 'coses', 'sikes', 'rarer', 'jiber', 'fixes', 'vines', 'volts', 'vases', 'farer', 'zests', 'waxes', 'jills', 'pumps', 'fazed', 'waves', 'janes', 'jests', 'vairs', 'sixes', 'vasts', 'wills', 'fazes', 'vests', 'sexes', 'bizes', 'hazes', 'barbs', 'sises', 'jades', 'sizes', 'dived', 'eases', 'river']
samples += ['veers', 'vests', 'keets', 'sises', 'widdy', 'rajes', 'bases', 'razes', 'jests', 'wises', 'river', 'nones', 'sales', 'fifes', 'razed', 'zests', 'sates', 'gases', 'eaves', 'galls', 'sades', 'sells', 'texts', 'torts', 'salal', 'rarer', 'tater', 'rares', 'sanes', 'sakes', 'janes', 'rared', 'tilts', 'lulus', 'doses', 'tates', 'sores', 'waded', 'cases', 'saves', 'cozes', 'rases', 'watts', 'saxes', 'sails', 'zaxes', 'seeps', 'sikes', 'sabes', 'pumps', 'sizes', 'vills', 'batts', 'fixes', 'gaged', 'tight', 'sixes', 'roses', 'tests', 'eaved', 'nines', 'fazes', 'gages', 'eases', 'hazes', 'sones', 'setts', 'fifed', 'kinks', 'seeds', 'faxed', 'parer', 'vases', 'waves', 'soles', 'loses', 'coses', 'faded', 'sines', 'sears']
samples += ['setts', 'waxes', 'babes', 'tyees', 'farer', 'vales', 'satis', 'raper', 'caves', 'haves', 'sices', 'seeks', 'buses']
samples += ['razes', 'lases', 'gaged', 'rared', 'texts', 'eaves', 'sears', 'caves', 'kinks', 'vills', 'rases', 'cozes', 'faxed', 'zaxes', 'tests', 'wises', 'gases', 'fifes', 'raper', 'sices', 'coses', 'sikes', 'gages', 'rarer', 'rares', 'setts', 'fixes', 'vases', 'farer', 'faded', 'zests', 'waxes', 'pumps', 'waves', 'janes', 'keets', 'jests', 'sixes', 'fazes', 'vests', 'hazes', 'sises', 'jades', 'sizes', 'razed', 'eases', 'lulus', 'river']
samples += ['jills', 'wives', 'sects']
samples +=  ['minks', 'zoner', 'yaffs', 'rases', 'mimes', 'pixes', 'noons', 'zooks', 'wared', 'razer', 'pipes', 'jades', 'mozos', 'eaved', 'sells', 'mooed', 'joked', 'surfs', 'jocks', 'coxed', 'mazed', 'micks', 'bases', 'mowed', 'wises', 'socks', 'gyves', 'hades', 'pests', 'waves', 'vines', 'dozer', 'howes', 'wipes', 'dozes', 'hared', 'zills', 'wowed', 'voles', 'hived', 'moved', 'hazes', 'wived', 'wakes', 'gibed', 'egged', 'jills', 'fuzed', 'vests', 'mises', 'eases', 'sooks', 'vowed', 'sicks', 'sinhs', 'sines', 'jaded', 'molly', 'mixed', 'mewed', 'sulus', 'sumps', 'jibed', 'sises', 'sizes', 'meets', 'foxes', 'haves', 'goxes', 'coxes', 'wades', 'fezes', 'kinks', 'syces', 'vills', 'boxed', 'sakes', 'waxes', 'jives', 'sills', 'hided', 'yipes', 'hajes', 'sipes', 'zaxes', 'rover', 'jiber', 'sanes', 'lalls', 'rower', 'zones', 'wanes', 'saxes', 'jived', 'fuzes', 'sinks', 'vexes', 'hiker', 'wider', 'rared', 'sikes', 'hides', 'pines', 'joker', 'vined', 'vomer', 'woods']
samples = set(samples)
print(len(samples), ','.join(samples))
# debug = False
# for word in samples:
#     w = Wordle(word, verbose=debug)
#     if debug:
#         print('WORD IS ' + word)
#     got_ans, cands = solve_wordle(w, settings={
#         'use_conditional': True,
#         'non_strict': True,
#         'use_pos': True
#     }, debug=debug)
#     if debug:
#         print('\n\n')
#     if not got_ans:
#         if debug:
#             print(f'Couldn\'t solve for {word}. Cands: {len(cands)} - {cands}')
#         fails.append((word, cands))
# print([f[0] for f in fails], len(fails), len(samples))


# None - 11 - ['wares', 'shoos', 'gains', 'sware', 'jades', 'worms', 'slaws', 'wrier', 'ricer', 'kikes', 'mater']
# Conditional - 7 - ['wares', 'sware', 'jades', 'worms', 'slaws', 'ricer', 'kikes']
# Non-strict - 3 - ['varia', 'jades', 'siver']

# positiojn based - (6, ['eaved', 'zills', 'molls', 'sises', 'vests', 'sumps'])

# Test pos-based. Non pos-based 62/170, pos-based 36/170


237 razes,pines,sakes,mooed,jived,putts,cases,raver,sones,haves,wanes,noons,saxes,lases,gaged,loses,dozes,emeer,jocks,rared,syces,minks,songs,texts,wades,veers,jaded,zoner,joker,eaves,wived,polos,sears,jakes,buses,vales,micks,mozos,molls,wives,sipes,winds,sicks,hared,caves,goxes,tarts,parer,kinks,batts,vills,saves,rases,solos,rover,rower,sills,zooks,tyees,fuses,coxed,cozes,wared,watts,fifed,wowed,faxed,oozed,jibes,sabes,oozes,wakes,hajes,nines,joked,pawer,coxes,vanes,sinhs,zaxes,kanes,emmer,haded,tests,sells,pills,veeps,yaffs,vexer,zarfs,ricer,javas,tight,hiker,wises,welly,bases,samps,gases,fifes,moved,lalls,widdy,winks,raper,sices,sores,pipes,sanes,sulus,bises,zills,sails,mises,hided,coses,sikes,eaved,vexed,tater,salal,vined,gages,septs,sates,rarer,rares,setts,jokes,babes,faves,pekes,surfs,vexes,pixes,jiber,gibed,punks,stats,doses,dozer,jives,jowed,fuzed,molly,fixes,riper,vines,wipes,razer,howes,seeds,torts,seeks,volts,vases,sades,mixed,boxed,farer,faded,zests,waxes,jills,pumps,foxes,

In [814]:
fails = []
K = len(five_words)
words = random.sample(five_words, K)
for x in range(K):
    if (x+1) % 10 == 0:
        print(f'iter {x}:\tfailed: {len(fails)}\taccuracy:{(1 - len(fails)/x)*100:.02f}%')
    word = words[x]
    w = Wordle(word, verbose=False)
    got_ans, cands = solve_wordle(w, settings={
        'use_conditional': True,
        'non_strict': True
    }, debug=False)
    if not got_ans:
#         print(f'Couldn\'t solve for {word}')
        fails.append(word)
len(fails), fails

iter 9:	failed: 0	accuracy:100.00%
iter 19:	failed: 0	accuracy:100.00%
iter 29:	failed: 0	accuracy:100.00%
iter 39:	failed: 0	accuracy:100.00%
iter 49:	failed: 0	accuracy:100.00%
iter 59:	failed: 0	accuracy:100.00%
iter 69:	failed: 0	accuracy:100.00%
iter 79:	failed: 0	accuracy:100.00%
iter 89:	failed: 0	accuracy:100.00%
iter 99:	failed: 0	accuracy:100.00%
iter 109:	failed: 0	accuracy:100.00%
iter 119:	failed: 0	accuracy:100.00%
iter 129:	failed: 0	accuracy:100.00%
iter 139:	failed: 0	accuracy:100.00%
iter 149:	failed: 0	accuracy:100.00%
iter 159:	failed: 0	accuracy:100.00%
iter 169:	failed: 0	accuracy:100.00%
iter 179:	failed: 0	accuracy:100.00%
iter 189:	failed: 1	accuracy:99.47%
iter 199:	failed: 2	accuracy:98.99%
iter 209:	failed: 2	accuracy:99.04%
iter 219:	failed: 2	accuracy:99.09%
iter 229:	failed: 2	accuracy:99.13%
iter 239:	failed: 2	accuracy:99.16%
iter 249:	failed: 2	accuracy:99.20%
iter 259:	failed: 2	accuracy:99.23%
iter 269:	failed: 2	accuracy:99.26%
iter 279:	failed: 2	a

iter 2229:	failed: 13	accuracy:99.42%
iter 2239:	failed: 13	accuracy:99.42%
iter 2249:	failed: 13	accuracy:99.42%
iter 2259:	failed: 13	accuracy:99.42%
iter 2269:	failed: 13	accuracy:99.43%
iter 2279:	failed: 13	accuracy:99.43%
iter 2289:	failed: 13	accuracy:99.43%
iter 2299:	failed: 13	accuracy:99.43%
iter 2309:	failed: 13	accuracy:99.44%
iter 2319:	failed: 13	accuracy:99.44%
iter 2329:	failed: 13	accuracy:99.44%
iter 2339:	failed: 13	accuracy:99.44%
iter 2349:	failed: 13	accuracy:99.45%
iter 2359:	failed: 13	accuracy:99.45%
iter 2369:	failed: 13	accuracy:99.45%
iter 2379:	failed: 13	accuracy:99.45%
iter 2389:	failed: 14	accuracy:99.41%
iter 2399:	failed: 14	accuracy:99.42%
iter 2409:	failed: 14	accuracy:99.42%
iter 2419:	failed: 14	accuracy:99.42%
iter 2429:	failed: 14	accuracy:99.42%
iter 2439:	failed: 14	accuracy:99.43%
iter 2449:	failed: 14	accuracy:99.43%
iter 2459:	failed: 16	accuracy:99.35%
iter 2469:	failed: 16	accuracy:99.35%
iter 2479:	failed: 16	accuracy:99.35%
iter 2489:	f

iter 4389:	failed: 29	accuracy:99.34%
iter 4399:	failed: 29	accuracy:99.34%
iter 4409:	failed: 29	accuracy:99.34%
iter 4419:	failed: 29	accuracy:99.34%
iter 4429:	failed: 29	accuracy:99.35%
iter 4439:	failed: 30	accuracy:99.32%
iter 4449:	failed: 30	accuracy:99.33%
iter 4459:	failed: 30	accuracy:99.33%
iter 4469:	failed: 30	accuracy:99.33%
iter 4479:	failed: 30	accuracy:99.33%
iter 4489:	failed: 30	accuracy:99.33%
iter 4499:	failed: 30	accuracy:99.33%
iter 4509:	failed: 30	accuracy:99.33%
iter 4519:	failed: 30	accuracy:99.34%
iter 4529:	failed: 30	accuracy:99.34%
iter 4539:	failed: 30	accuracy:99.34%
iter 4549:	failed: 30	accuracy:99.34%
iter 4559:	failed: 30	accuracy:99.34%
iter 4569:	failed: 30	accuracy:99.34%
iter 4579:	failed: 30	accuracy:99.34%
iter 4589:	failed: 30	accuracy:99.35%
iter 4599:	failed: 30	accuracy:99.35%
iter 4609:	failed: 30	accuracy:99.35%
iter 4619:	failed: 30	accuracy:99.35%
iter 4629:	failed: 30	accuracy:99.35%
iter 4639:	failed: 30	accuracy:99.35%
iter 4649:	f

iter 6549:	failed: 45	accuracy:99.31%
iter 6559:	failed: 45	accuracy:99.31%
iter 6569:	failed: 45	accuracy:99.31%
iter 6579:	failed: 45	accuracy:99.32%
iter 6589:	failed: 45	accuracy:99.32%
iter 6599:	failed: 45	accuracy:99.32%
iter 6609:	failed: 45	accuracy:99.32%
iter 6619:	failed: 45	accuracy:99.32%
iter 6629:	failed: 45	accuracy:99.32%
iter 6639:	failed: 45	accuracy:99.32%
iter 6649:	failed: 45	accuracy:99.32%
iter 6659:	failed: 45	accuracy:99.32%
iter 6669:	failed: 46	accuracy:99.31%
iter 6679:	failed: 46	accuracy:99.31%
iter 6689:	failed: 46	accuracy:99.31%
iter 6699:	failed: 46	accuracy:99.31%
iter 6709:	failed: 46	accuracy:99.31%
iter 6719:	failed: 46	accuracy:99.32%
iter 6729:	failed: 46	accuracy:99.32%
iter 6739:	failed: 46	accuracy:99.32%
iter 6749:	failed: 46	accuracy:99.32%
iter 6759:	failed: 46	accuracy:99.32%
iter 6769:	failed: 47	accuracy:99.31%
iter 6779:	failed: 47	accuracy:99.31%
iter 6789:	failed: 47	accuracy:99.31%
iter 6799:	failed: 47	accuracy:99.31%
iter 6809:	f

(62,
 ['vills',
  'oozed',
  'sabes',
  'sixes',
  'bises',
  'jived',
  'winds',
  'wills',
  'fifes',
  'joker',
  'baked',
  'zaxes',
  'eaves',
  'rears',
  'river',
  'jakes',
  'molls',
  'dozes',
  'zills',
  'sises',
  'wises',
  'pills',
  'jiber',
  'vanes',
  'punks',
  'lolly',
  'eases',
  'jills',
  'waxes',
  'welly',
  'zests',
  'jowed',
  'seeps',
  'jokes',
  'saxes',
  'sills',
  'saves',
  'wives',
  'fuses',
  'faded',
  'pekes',
  'solos',
  'jibes',
  'poops',
  'winks',
  'eaved',
  'sumps',
  'sipes',
  'putts',
  'riper',
  'bases',
  'gills',
  'oozes',
  'haded',
  'vests',
  'polos',
  'fazes',
  'fifed',
  'rover',
  'sizes',
  'vexed',
  'songs'])

In [729]:
pprint(sorted([(f[0], len(f[1])) for f in fails], key=lambda x: -x[1]))
# Counter([len(f[1]) for f in fails]).most_common()

[('texts', 5),
 ('eaves', 5),
 ('sears', 5),
 ('vills', 5),
 ('zaxes', 5),
 ('tests', 5),
 ('wises', 5),
 ('sices', 5),
 ('vases', 5),
 ('zests', 5),
 ('waxes', 5),
 ('jills', 5),
 ('waves', 5),
 ('jests', 5),
 ('rears', 5),
 ('sixes', 5),
 ('wills', 5),
 ('fazes', 5),
 ('vests', 5),
 ('sises', 5),
 ('sizes', 5),
 ('eases', 5),
 ('raver', 4),
 ('wanes', 4),
 ('jakes', 4),
 ('wakes', 4),
 ('rarer', 4),
 ('vines', 4),
 ('farer', 4),
 ('janes', 4),
 ('sacks', 4),
 ('sexes', 4),
 ('wades', 3),
 ('wived', 3),
 ('caves', 3),
 ('goxes', 3),
 ('cozes', 3),
 ('faxed', 3),
 ('veeps', 3),
 ('bises', 3),
 ('coses', 3),
 ('setts', 3),
 ('pumps', 3),
 ('fazed', 3),
 ('bizes', 3),
 ('hazes', 3),
 ('jades', 3),
 ('dived', 3),
 ('lases', 2),
 ('gaged', 2),
 ('rared', 2),
 ('wives', 2),
 ('sipes', 2),
 ('kinks', 2),
 ('rases', 2),
 ('oozed', 2),
 ('zarfs', 2),
 ('ricer', 2),
 ('javas', 2),
 ('gases', 2),
 ('fifes', 2),
 ('raper', 2),
 ('sikes', 2),
 ('jiber', 2),
 ('fixes', 2),
 ('volts', 2),
 ('vairs',

In [815]:
str([f for f in fails])

"['vills', 'oozed', 'sabes', 'sixes', 'bises', 'jived', 'winds', 'wills', 'fifes', 'joker', 'baked', 'zaxes', 'eaves', 'rears', 'river', 'jakes', 'molls', 'dozes', 'zills', 'sises', 'wises', 'pills', 'jiber', 'vanes', 'punks', 'lolly', 'eases', 'jills', 'waxes', 'welly', 'zests', 'jowed', 'seeps', 'jokes', 'saxes', 'sills', 'saves', 'wives', 'fuses', 'faded', 'pekes', 'solos', 'jibes', 'poops', 'winks', 'eaved', 'sumps', 'sipes', 'putts', 'riper', 'bases', 'gills', 'oozes', 'haded', 'vests', 'polos', 'fazes', 'fifed', 'rover', 'sizes', 'vexed', 'songs']"

In [270]:
str('gorge, wares, shoos, gains, sware, varia, casks, jades, worms, slaws, zaman, finns, vinny'.split(', '))

"['gorge', 'wares', 'shoos', 'gains', 'sware', 'varia', 'casks', 'jades', 'worms', 'slaws', 'zaman', 'finns', 'vinny']"

In [579]:
word = 'quick' # random.choice(five_words)
print(f'Instantiating Wordle with the word [{word}]:')
w = Wordle(word, verbose=True)
clues = []
for i in range(Wordle.MAX_GUESSES):
    chosen, cands, numcands = solve_guesser(clues, settings=DEFAULT_SETTINGS, debug=True)
    print(f'Choosing [{chosen}]. Total {numcands} candidates: {cands}...')
    clue, state = w.guess(chosen)
    if clue:
        clues.append((chosen, clue))

Instantiating Wordle with the word [quick]:
defaultdict(<class 'set'>, {}) defaultdict(<class 'set'>, {}) set()
explore cands  [('arose', -18514), ('arise', -18189), ('raise', -18189), ('serai', -18189), ('aloes', -18051), ('arles', -18014), ('earls', -18014), ('lares', -18014), ('laser', -18014), ('lears', -18014)]
Choosing [arose]. Total 8636 candidates: ['arose', 'arise', 'raise', 'serai', 'aloes']...
⬛⬛⬛⬛⬛
defaultdict(<class 'set'>, {}) defaultdict(<class 'set'>, {}) {'r', 'e', 's', 'a', 'o'}
explore cands  [('unity', -1014), ('linty', -978), ('unlit', -976), ('until', -976), ('lytic', -955), ('culti', -953), ('cutin', -940), ('tunic', -940), ('unify', -937), ('tulip', -926)]
Choosing [unity]. Total 409 candidates: ['unity', 'linty', 'unlit', 'until', 'lytic']...
🟨⬛🟩⬛⬛
defaultdict(<class 'set'>, {'i': {2}}) defaultdict(<class 'set'>, {'u': {0}}) {'r', 'e', 's', 'y', 't', 'a', 'n', 'o'}
conditional unknown freq Unknown: [0, 1, 3, 4]
Conditional unknown freq:26
Conditional:[('b', 1),

IndexError: list index out of range

In [534]:
# Evil wordle
# solve_guesser([], debug=True)
solve_guesser([('unlit', [0, 0, 0, 0, 0]), ('arose', [1, 0, 0, 1, 1]), ('dumpy', [0, 0, 0, 0, 0]), ('whack', [0, 0, 1, 0, 0]), ('befog', [0, 1, 1, 0, 0])], debug=True)
# solve_guesser([('arose', [0, 0, 0, 1, 1]), ('islet', [0, 1, 0, 1, 0]), ('pends', [0, 2, 0, 2, 2]), ('fumes', [0, 0, 0, 1, 2]), ('chews', [0, 0, 2, 0, 2])], debug=True)
# solve_guesser([('arose', [0, 0, 0, 1, 1]), ('unlit', [0, 0, 0, 0, 0]), ('pecky', [0, 2, 0, 0, 0])], debug=True)
# solve_guesser([('arose', [0, 0, 0, 1, 1]), ('unlit', [0, 0, 0, 0, 0]), ('pecky', [0, 2, 0, 0, 0]),
#                ('mixed', [0, 0, 0, 1, 1]), ('howfs', [0, 0, 0, 0, 2]), ('deeds', [0, 2, 2, 2, 2])], debug=True)

defaultdict(<class 'set'>, {}) defaultdict(<class 'set'>, {'a': {0, 2}, 's': {3}, 'e': {1, 4}, 'f': {2}}) {'w', 'p', 'm', 'b', 'r', 'c', 'k', 'h', 'l', 'y', 'g', 'u', 'd', 't', 'i', 'n', 'o'}


('faves', ['faves', 'faxes', 'fazes'], 3)

In [ ]:
# defaultdict(<class 'set'>, {'s': {4}}) defaultdict(<class 'set'>, {'o': {2}, 's': {3}, 't': {4}, 'f': {0}}) {'p', 'm', 'r', 'c', 'e', 'k', 'h', 'l', 'y', 'u', 'i', 'n', 'a'}
# conditional unknown freq  [0, 1, 2, 3] 26 defaultdict(<class 'int'>, {'o': 0.5, 's': 0.5, 't': 0.5, 'f': 0.5, 'd': 0, 'h': 0, 'e': 0, 'a': 0, 'l': 0, 'i': 0, 'r': 0, 'g': 0, 'b': 0, 'c': 0, 'k': 0, 'p': 0, 'm': 0, 'y': 0, 'n': 0, 'u': 0, 'v': 0, 'z': 0, 'x': 0, 'w': 0, 'j': 0, 'q': 0}) ['softs', 'toffs', 'tofts'] 3 0
# explore cands [('fatso', -2.0), ('foist', -2.0), ('fonts', -2.0), ('foots', -2.0), ('forts', -2.0), ('frost', -2.0), ('lofts', -2.0), ('softa', -2.0), ('softs', -2.0), ('softy', -2.0)]
# Choosing [fatso]. Total 3 candidates: ['softs', 'toffs', 'tofts']
# softs
# toffs
# tofts


In [739]:
solve_guesser([('arose', [0, 2, 0, 0, 0]), ('tulip', [0, 0, 0, 1, 0]), ('gunky', [0, 0, 1, 1, 0])], debug=True)


Right: _r___ Wrong: [('i', {3}), ('n', {2}), ('k', {3})] Absent: peslgyutao


('brink', ['brink', 'drink'], 2)